## TODO
~~only consider those that has @ as initial char in Citation columns~~   
~~add one column per tematic~~  
add column with comparison between old tile and new title  
~~only maintain N_index, from previous df~~  
~~check how parse bibtex, or do it manually (first prefered due to some latex in the text)~~~  
~~develop a function to do tematics one by one, in that way I can add manually new tematics without reload everything~~   
check ffmode else weird syntaxis in the entire text. Example: Acta BotÄ±fmmode\acutea\elseÃ¡\finica VenezuÄ±fmmode\acutee\elseÃ©\filica  
check duplicates and remove them  


Ä±fmmode\\(\w+)\\else.*\\fi

In [16]:
import pandas as pd
import re 
import numpy as np
import os.path

import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import homogenize_latex_encoding
from bibtexparser.customization import convert_to_unicode

#define parser  method for bibtex
bib_parser = BibTexParser()
bib_parser.customization = convert_to_unicode
# bib_parser.customization = homogenize_latex_encoding

from fuzzywuzzy import fuzz

## Load database

In [49]:
# load abstracts database
df_raw = pd.read_csv("../4_Abstract_extraction_Manual/DATABASE - DATABASE_preview.csv", low_memory = True)

In [50]:
df_raw

,N_Index,Old_title,Citation,Google link,Taxonomy,Systematic,Mexico,Unnamed: 7,Unnamed: 8,First gen.,...,Parsimony,Likelihood,Bayesian,MSC,Unnamed: 18,Notas,Autocheck: Abstract integrity (7),Autocheck: Abbrev. (22),Autocheck: Short abstract (22),Carlos doublecheck (170)
0,0,Andean-centred' genera in the short-branch cla...,"@article{Pirie2006Jan,\n author = {Piri...",https://scholar.google.com/scholar?&q=%27Andea...,1,1,0,1,0,1,...,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Mediterranean' disjunction in Senecio sect. Se...,no existe,https://scholar.google.com/scholar?&q=%27Medit...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
2,4,What are natural clades in the Caesalpinia gro...,no existe,https://scholar.google.com/scholar?&q=%27What+...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
3,18,5S NTS of chili peppers (Capsicum-Solanaceae):...,no existe,https://scholar.google.com/scholar?&q=5S+NTS+o...,Auto,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4,23,A 10-gene phylogeny of Solanum section Herpyst...,"@article{Tepe2011Aug,\n author = {Tepe,...",https://scholar.google.com/scholar?&q=A+10-gen...,0,1,0,1,0,1,...,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4986,12153,rbcL sequences provide phylogenetic insights a...,"@article{Haufler1995Oct,\n author = {Ha...",https://scholar.google.com/scholar?&q=rbcL+seq...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4987,12154,rbcL sequences: a promising tool for the molec...,"@article{Dubuisson1997Oct,\n author = {...",https://scholar.google.com/scholar?&q=rbcL+seq...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4988,12163,trnL-F and rpl16 Sequence Data and Dense Taxon...,"@article{Del-Pino2009Mar,\n author = {D...",https://scholar.google.com/scholar?&q=trnL-F+a...,?,Auto,?,?,?,Auto,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN
4989,12164,"ural Delineation, Molecular Phylogeny and Flor...","@article{Roquet2008Mar,\n author = {Roq...",https://scholar.google.com/scholar?&q=ural+Del...,?,Auto,?,?,?,?,...,?,?,?,?,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# list of markers for detec sanger, nuclear, and chloroplast info
nuclear_markers = "Internal transcribed spacer|ITS1|ITS2|ETS|nrITS|18S|59S|5.8s|GBSSI|COSII|25S|Ypr10|pgiC|ETS|ADH2F|ADH3R|Aat|Skdh|Pgi.?2|Tpi.?1|Tpi.?2|AFLP"
chl_markers = "trnL|trnG|trnR|trnT|trnS|trnF|trnH|trnQ|trnD|trnY|trnE|atpB|rubisco|rbcL|psbA|matk|psbJ|petA|ndhF|psaA|ycf3|ycf4|rpl16|trnSfM|trnfM|rps4|cemA|psbM|trnDGUC"


               
# set catagories of search
categories = {
    "Mexico": "(?<!new)(?<!nuevo)\s+m.[xj]ic",
    "Morphology": "morpholog|morfolog",
    "First gen.": f"{nuclear_markers}|{chl_markers}| sanger | marker| nuclear region| chloroplast region|regi.n.*? nucr|regi.n.*? cloro| spacer|AFLP|Microsat.ll?ite",
    "Second gen.": "radseq|angiosperm353|hybseq|454|illumina|iontorrent|whole genome|plastome",
    "Third gen.": "pacbio|nanopore",
    "Nuclear info": f"nuclear|{nuclear_markers}",
    "Chloroplast info": f"c.?lorop|{chl_markers}",
    "Parsimony": "parsimon",
    "Maximum likelihood": " ML |maximum likelihood|verosimilitud",
    "Bayesian": "bayes| beast",
    "MSC": " astral | astrid | bpp |coalescent",
    "New species": "new species|nueva. especie",
    "New genus": "new genus|nueva.? g.nero",
}

In [52]:
test = "A Molecular Phylogenetic Study of the Palmae (Arecaceae) Based on atpB, rbcL, and  nrDNA Sequences"
for k in categories:
    print(f"{k}: {re.search(categories[k], test, re.IGNORECASE)}")

Mexico: None
Morphology: None
First gen.: <re.Match object; span=(66, 70), match='atpB'>
Second gen.: None
Third gen.: None
Nuclear info: None
Chloroplast info: <re.Match object; span=(66, 70), match='atpB'>
Parsimony: None
Maximum likelihood: None
Bayesian: None
MSC: None
New species: None
New genus: None


In [53]:
#copy original dataframe (only first three columns)
df = df_raw.iloc[:, 0:3].copy()

#remove rows that has no bibtex citation
df = df[df["Citation"].str.startswith("@")]

In [54]:
#create final df, in which tematic columns will be added.
#in this dataframe will be the information for every citation field

final_df = pd.DataFrame()


for row in df.itertuples():
    new_row_values = {}
# for row in df.head(5).itertuples():
    #parse bibtex
    citation = bibtexparser.loads(row.Citation, bib_parser)
    citation = citation.entries[-1]
    
    #add N_Index 
    # row_values["N_Index"] = row.N_Index

    #add columns for each citation field (added prefix c. = citation)
    for field in citation:
        # #verify weird syntax for special chars 
        # citation[field] = re.sub(r"Ä±fmmode\\(\w+)\\else.*\\fi", r"\1", citation[field])


        new_row_values[f"c.{field.lower()}"] = citation[field]

    #count names and add them to the final df
    ##split names by " and "
    try:
        names = citation["author"].split(" and ")
        authors = bibtexparser.customization.getnames(names)
        n_authors = len(authors)
    except:
        authors = "ERROR"
        n_authors = "ERROR"
    
    #replace names with a organized structure from bibtexparser
    new_row_values["c.author"] = authors
    new_row_values["s.n_authors"] = str(n_authors) #in this prefix s. is added to refer statistics

    #add old title
    new_row_values["a.oldtitle"] = row.Old_title #prefix a. is for archive
    
    #compare old and new title
    try:
        titles_similitud = fuzz.token_sort_ratio(row.Old_title.lower(), citation["title"].lower())
        new_row_values["v.simil_titles"] = titles_similitud #prefix v. is for verification
    except:
        new_row_values["v.simil_titles"] = 0

    row_to_add = pd.Series(new_row_values, name=row.N_Index)
    final_df = final_df.append(row_to_add, sort=True)

In [55]:
final_df

,a.oldtitle,c.abstract,c.address,c.author,c.booktitle,c.doi,c.entrytype,c.eprint,c.id,c.isbn,...,c.note,c.number,c.pages,c.publisher,c.title,c.url,c.volume,c.year,s.n_authors,v.simil_titles
0,Andean-centred' genera in the short-branch cla...,Aim We test biogeographical hypotheses regardi...,NaN,"[Pirie, Michael D., Chatrou, Lars W., Mols, Jo...",NaN,10.1111/j.1365-2699.2005.01388.x,article,NaN,Pirie2006Jan,NaN,...,NaN,1,31--46,"John Wiley & Sons, Ltd",`Andean-centred′ genera in the short-branch cl...,NaN,33,2006,5,100.0
23,A 10-gene phylogeny of Solanum section Herpyst...,Premise of the study: Solanum section Herpysti...,NaN,"[Tepe, Eric J., Farruggia, Frank T., Bohs, Lynn]",NaN,10.3732/ajb.1000516,article,NaN,Tepe2011Aug,NaN,...,NaN,8,1356--1365,"John Wiley & Sons, Ltd",A 10-gene phylogeny of Solanum section Herpyst...,NaN,98,2011,3,100.0
26,A 250 plastome phylogeny of the grass family (...,The systematics of grasses has advanced throug...,NaN,"[Saarela, Jeffery M., Burke, Sean V., Wysocki,...",NaN,10.7717/peerj.4299,article,NaN,Saarela2018Feb,NaN,...,NaN,NaN,e4299,PeerJ Inc.,A 250 plastome phylogeny of the grass family (...,NaN,6,2018,10,100.0
29,A 313 plastome phylogenomic analysis of Pooide...,"In this study, we analyzed 313 plastid genomes...",NaN,"[Orton, Lauren M., Barberıfmmode\acutea\elseá\...",NaN,10.1016/j.ympev.2021.107110,article,NaN,Orton2021Jun,NaN,...,NaN,NaN,107110,Academic Press,A 313 plastome phylogenomic analysis of Pooide...,NaN,159,2021,7,100.0
32,A Chloroplast Phylogeny of Agavaceae subfamily...,Abstractıfmmode---\else—\fi Agavaceae subfamil...,NaN,"[Halpin, Kate M., Fishbein, Mark]",NaN,10.1600/036364413X674850,article,NaN,Halpin2013Dec,NaN,...,NaN,4,996--1011,American Society of Plant Taxonomists,A Chloroplast Phylogeny of Agavaceae subfamily...,NaN,38,2013,2,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12153,rbcL sequences provide phylogenetic insights a...,Opinions on the phylogeny and classification o...,NaN,"[Haufler, Christopher H., Ranker, Tom A.]",NaN,NaN,article,NaN,Haufler1995Oct,NaN,...,NaN,4,361--374,American Fern Society,RbcL Sequences Provide Phylogenetic Insights a...,http://www.jstor.org/stable/1547815,85,1995,2,100.0
12154,rbcL sequences: a promising tool for the molec...,A phylogenetic study usingrbcL sequences was p...,NaN,"[Dubuisson, J.-Y.]",NaN,10.1006/mpev.1997.0414,article,NaN,Dubuisson1997Oct,NaN,...,NaN,2,128--138,Academic Press,rbcL Sequences: A Promising Tool for the Molec...,NaN,8,1997,1,89.0
12163,trnL-F and rpl16 Sequence Data and Dense Taxon...,The Amaranthaceae-Chenopodiaceae alliance has ...,NaN,"[Del-Pino, Ivonne Sıfmmode\acutea\elseá\finche...",NaN,10.1600/036364409787602401,article,NaN,Del-Pino2009Mar,NaN,...,NaN,1,57--67,American Society of Plant Taxonomists,trnL-F and rpl16 Sequence Data and Dense Taxon...,NaN,34,2009,3,100.0
12164,"ural Delineation, Molecular Phylogeny and Flor...",The circumscription and infrageneric classific...,NaN,"[Roquet, Cristina, Sıfmmode\acutea\elseá\fiez,...",NaN,10.1600/036364408783887465,article,NaN,Roquet2008Mar,NaN,...,NaN,1,203--217,American Society of Plant Taxonomists,"Natural Delineation, Molecular Phylogeny and F...",NaN,33,2008,6,91.0


In [56]:
# export database as csv
final_df.to_csv("bare_final_database.csv")

## Definition function to search tematics

In [57]:
# Function definition for tematic addition
def add_tematic_column(file, regex, fields_to_search, search_title, overwrite=True, sep=",", index_col=0):
    """
    Open database and search reges in fields_to_search and append column
    """
    # load database
    df = pd.read_csv(file, low_memory = True, delimiter=sep, index_col=index_col)

    # code title
    search_title = f"t.{search_title}"
    search_title= re.sub("\s", "_", search_title)

    # create empty column for result
    df[search_title] = np.nan

    # iterate row by row
    for index, row in df.iterrows(): 
        # search every pattern in title
        for field in fields_to_search:
            try: 
                if re.search(regex, row[field], re.IGNORECASE):
                    #add Auto in the cell to indicate this part of the search is true
                    df.loc[index, search_title] = "1"
            except:
                continue
            
    if overwrite:
        df.to_csv(file)
        return f"Found {df[df[search_title].notnull()][search_title].shape[0]} for {search_title} regex ({regex}) in {fields_to_search}, {file} overwrote!"
    else:
        return df[df[search_title].notnull()][search_title]

In [58]:
#test function with Mexico regex
add_tematic_column("bare_final_database.csv", categories[list(categories.keys())[0]], ["c.abstract", "c.title"], list(categories.keys())[0])

"Found 252 for t.Mexico regex ((?<!new)(?<!nuevo)\\s+m.[xj]ic) in ['c.abstract', 'c.title'], bare_final_database.csv overwrote!"

## Add tematics

In [59]:
# iterate over ALL categories. Replace previous results!
for category in categories:
    add_tematic_column("bare_final_database.csv", categories[category], ["c.abstract", "c.title"], category)

In [60]:
#For individual tematics use add_tematic_column method

## Manual families extraction -- Should be replaced by another method

In [61]:
#Get families from abstracts

file = "bare_final_database.csv"
df = pd.read_csv(file, low_memory = True, index_col=0)

fields_to_search = ["c.abstract", "c.title"]

#Load list of families by major groups
with  open("../Data_ThePlantList/Angiosperms.txt", "r") as txtfile: 
    angiosperms = [line.strip() for line in txtfile]
    
with  open("../Data_ThePlantList/Bryophytes.txt", "r") as txtfile: 
    bryophytes = [line.strip() for line in txtfile]
    
with  open("../Data_ThePlantList/Gymnosperms.txt", "r") as txtfile: 
    gymnosperms = [line.strip() for line in txtfile]
    
with  open("../Data_ThePlantList/Pteridophytes.txt", "r") as txtfile: 
    pteridophytes = [line.strip() for line in txtfile]

all_families = angiosperms + bryophytes + gymnosperms + pteridophytes

for index, row in df.iterrows(): 
    families = []
    groups = []
    for field in fields_to_search:
        try:
#             [families.append(i.lower()) for i in re.findall("[A-Za-z]+(?<=ceae)", row[field], re.IGNORECASE) if i not in families]
            for family in re.findall("[A-Za-z]+(?<=ceae)", row[field], re.IGNORECASE):
                family_to_add = ""
                #check family and correct
                family_corrected = [s for s in all_families if fuzz.token_sort_ratio(s, family) >= 80]
                if len(family_corrected) == 1:
                    family_to_add = family_corrected[0]
#                     print(family, "corrected", family_corrected[0])
                else:                    
                    family_to_add = family
#                     print(family, "no corrected")
                
                if family_to_add not in families:
                    families.append(family_to_add.lower())
                
                
                #check group
                if family_to_add in angiosperms:  df.loc[index, "t.angiosperms"] = 1
                elif family_to_add in bryophytes: df.loc[index, "t.bryophytes"] = 1
                elif family_to_add in gymnosperms: df.loc[index, "t.gymnosperms"] = 1
                elif family_to_add in pteridophytes: df.loc[index, "t.pteridophytes"] = 1
                

        except:
            continue
    
    #add to df
    list_of_families = ",".join(set(families))
    df.loc[index, "t.families"] = list_of_families

df.to_csv(file)

## Add type of journal (open source, scielo), country, Quartil and Impact factor (year based)

In [ ]:
# search journal in SCIMago databases and add info to the main database

file = "bare_final_database.csv"
df = pd.read_csv(file, low_memory = True, index_col=0)

#iterarte by year to open only once each big database
for year in range(1999, 2021):
    
    print(year)
        
    scimagoFULL_df = pd.read_csv(f"../Data_scimago/all_journals/scimagojr {year}.csv", low_memory = True, delimiter=";", index_col=0, error_bad_lines=False)
    scimagoOPEN_df = pd.read_csv(f"../Data_scimago/opensource_journals/scimagojr {year}.csv", low_memory = True, delimiter=";", index_col=0, error_bad_lines=False)
    scimagoSCIELO_df = pd.read_csv(f"../Data_scimago/scielo_journals/scimagojr {year}.csv", low_memory = True, delimiter=";", index_col=0, error_bad_lines=False)

    

    for index, row in df.iterrows():
        journal = row["c.journal"]
        paper_year = row["c.year"]

        #avoid nulls
        if isinstance(journal, str):

            #check quartil, impact factor and country from general database
            result = scimagoFULL_df[scimagoFULL_df["Title"].str.lower() == journal.lower()]

            if result.shape[0] > 0:
                # check year for information that change yearly
                if paper_year == year:
                    if not isinstance(result["SJR"].values[0], float):
                        _sjr = float(result["SJR"].values[0].replace(',', '.'))
                    else:
                        _sjr = result["SJR"].values[0]
                    df.loc[index, "s.sjr"] = _sjr
                    df.loc[index, "s.q"] = result["SJR Best Quartile"].values[0]
                    df.loc[index, "s.h"] =  result["H index"].values[0]
                    
                #other static information, just added
                df.loc[index, "s.country"] =  result["Country"].values[0]
                df.loc[index, "s.scimago"] = 1

            #check if exist in opensource databases (THIS COULD BE SENSITIVE TO YEAR TOO, CHECK IF CHANGES)
            result = scimagoOPEN_df[scimagoOPEN_df["Title"].str.lower() == journal.lower()]
            if result.shape[0] > 0:
                df.loc[index, "s.opensource"] = 1

                
            #check if exist in scielo database
            result = scimagoSCIELO_df[scimagoSCIELO_df["Title"].str.lower() == journal.lower()]
            if result.shape[0] > 0:
                df.loc[index, "s.scielo"] = 1


    
    df.to_csv(file)

## Add markers used

In [143]:
#load database
file = "bare_final_database.csv"
df = pd.read_csv(file, low_memory = True, index_col=0)

#define where to search
fields_to_search = ["c.abstract", "c.title"]

#define some common markers
markers = {
    "18S-26S": ["18S-26S"],
    "18S": ["[ (]18S"],
    "ADH2F-ADH3R": ["ADH2F-ADH3R"],
    "AFLPs": ["AFLP", "Amplified fragment length polymorphism"],
    "aptF-aptH": ["aptF-aptH"],
    "atpB-rbcL": ["atpB-rbcL"],
    "atpB": ["[ (]atpB"],
    "clpp": ["clpp"],
    "COSII": ["COSII"],
    "ETS": ["ETS"],
    "GBSSI": ["GBSSI"],
    "ITS": ["ITS", "ITS1-5.8s-ITS2", "ITS1", "5.8s", "ITS2", "Internal transcribed spacer", "59S"],
    "matK": ["matK"],
    "ndhA": ["ndhA"],
    "ndhF": ["ndhF"],
    "petB": ["petB"],
    "pgiC": ["pgiC"],
    "psaA-ycf3": ["psaA-ycf3"],
    "psaC-ndhE": ["psaC-ndhE"],
    "psaJ-rpl33": ["psaJ-rpl33"],
    "psbE-petL": ["psbE-petL"],
    "psbJ‐petA": ["psbJ‐petA"],
    "psbL-trnS": ["psbL-trnS[-AUGC]*"],
    "psbM-trnD": ["psbM-trnD[-AUGC]*"],
    "psbZ-trnG": ["psbZ-trnG[-AUGC]*"],
    "rbcL": ["[ (]rbcL","RuBisCO", "rubisco"],
    "rpl12-clpp": ["rpl12-clpp"],
    "rpl32-trnL": ["rpl32-trnL[-AUGC]*"],
    "rpoB-psbZ": ["rpoB[-AUGC]*-psbZ[-AUGC]*", "BZ"],
    "rpoB-trnC-GCA": ["rpoB-trnC[-AUGC]*"],
    "rpoC2-rpoC1": ["rpoC2-rpoC1"],
    "rps16-trnQ": ["rps16-trnQ[-AUGC]*"],
    "rps19": ["rps19"],
    "rps2-rpoc2": ["rps2-rpoc2"],
    "rps4-trnS ": ["rps4-trnS[-AUGC]*"],
    "trnC-ycf6": ["trnC[-AUGC]*-ycf6"],
    "trnD-psbM": ["trnD[-AUGC]*-psbM"],
    "trnD-trnT": ["trnD[-AUGC]*‐trnY[-AUGC]*‐trnE[-AUGC]*‐trnT[-AUGC]*"],
    "trnD-trnY": ["trnD-AUGC]*-trnY-AUGC]*"],
    "trnD‐trnY": ["trnD[-AUGC]*‐trnY[-AUGC]*"],
    "trnG-trnfM" : ["trnG[-AUGC]*-trnfM[-AUGC]*"],
    "trnH-psbA": ["trnH[-AUGC]*-psbA", "psbA-trnH[-AUGC]*"],
    "trnK-rps16": ["trnK[-AUGC]*-rps16"],
    "trnL-trnF": ["trnL[-AUGC]*-trnF[-AUGC]*", "trnL-F", "trnF-trnL"],
    "trnM-atpE": ["trnM[-AUGC]*-atpE"],
    "trnP-psaJ": ["trnP[-AUGC]*-psaJ"],
    "trnQ-psbK": ["trnQ[-AUGC]*-psbK[-AUGC]*"],
    "trnR-atpA": ["trnR[-AUGC]*-atpA"],
    "trnS-psbZ": ["trnS[-AUGC]*-psbZ[-AUGC]*"],
    "trnS-rps4": ["trnS[-AUGC]*-rps4"],
    "trnS-trnG": ["trnS[-AUGC]*-trnG[-AUGC]*"],
    "trnS‐trnfM ": ["trnS[-AUGC]*‐trnfM[-AUGC]*"],
    "trnT-trnE": ["trnT[-AUGC]*-trnE[-AUGC]*", "trnE[-AUGC]*‐trnT[-AUGC]*"],
    "trnT-trnF": ["trnT[-AUGC]*-trnF[-AUGC]*", "trnF[-AUGC]*‐trnT[-AUGC]*"],
    "trnT-trnL": ["trnT[-AUGC]*-trnL[-AUGC]*", "trnT-L"],
    "trnT-trntL": ["trnT[-AUGC]*-trntL[-AUGC]*"],
    "trnY‐trnE": ["trnY[-AUGC]*‐trnE[-AUGC]*"],
    "ycf1": ["ycf1"],
    "ycf15-ycf1": ["ycf15-ycf1"],
    "ycf4-cemA": ["ycf4-cemA"],
    "Ypr10": ["Ypr10"],
    "Microsatellites": ["Microsat.ll?ite"],
          }

#iterate row by row
for index, row in df.iterrows(): 
    #create empty result for each row
    result_markers = []
    #iterate for all markers in dictionary
    for marker in markers.keys():
        
        #search in each field defined
        for field in fields_to_search:
            #replace all ndash and mdash to normal dash en each field
            string_field = row[field]
            for r in (("–", "-"), ("—", "-")):
                if isinstance(string_field, str):
                    string_field = string_field.replace(*r)
                
            #join string_fieldin all synonyms for a given marker
            regex_string = "|".join(markers[marker])
            #search for all elements
            try:
                search = re.findall(regex_string, string_field)

                if search:
                    result_markers.append(marker)
            except:
                continue

    if result_markers:
        df.loc[index, "t.markers"] = ",".join(set(result_markers))
    else:
        df.loc[index, "t.markers"] = np.nan

                
df.to_csv(file)

## Add species per study and check mexican endmics

In [64]:
# using a.oldtitle search in the original databases in 1 and 1b.
# Iterate each row in bare_final_database against every species in both 1 and 1b files
# I item in a.oldtitle is found in Paper titles, add "Species after GB analysis" t.sp_gb to a columns and add "Searched name" to an independent column (t.species) (the latter is the Villaseñor species name)
# additionally check with Entrez to get family and other info about names.

In [ ]:
#load database
file = "bare_final_database.csv"
df = pd.read_csv(file, low_memory = True, index_col=0)

#put endemic species into a list
with open("../Data_endemicSpeciesMexico/endemics_uniques.txt", "r") as f:
    data = f.read()
    endemic_mx = data.splitlines()

list_endemic_species = []

for index, row in df.iterrows():
    result_species = []
    result_species_endemic = []

    #open raw database file
    with open("../1_Datataxa_extraction/FIRSTTEST.csv", mode="r") as file:
        #load lines
        lines = file.readlines()
        #iterate over each row (aka species)
        for line in lines[1:]:
            publications = None
            species = None
            #manually parse each column
            cells = line.split("\",\"")
            if len(cells) >= 2:
                publications = cells[2].split("|")
            if len(cells) >= 5:
                species = cells[5][:-2]
            
            if publications:
                #Check if old title is in publications in raw database
                if row["a.oldtitle"] in publications:
                    result_species.append(species)
                    if species in endemic_mx:
                        result_species_endemic.append(species)
                        list_endemic_species.append(species)
                        df.loc[index, "s.includedEndemicMXSpp"] = 1
    
    df.loc[index, "t.species"] = ",".join(result_species)
    df.loc[index, "t.endemicMXspecies"] = ",".join(result_species_endemic)

df.to_csv(file)

In [ ]:
file = "bare_final_database.csv"
df = pd.read_csv(file, low_memory = True, index_col=0)

In [ ]:
df["t.species","t.endemicMXspecies", "s.includedEndemicMXSpp"]]